In [1]:
import numpy as np
import pandas

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import csv

In [2]:
from uci_datasets import Dataset

In [3]:
data_name = "slice"
data = Dataset(data_name)

slice dataset, N=53500, d=385


In [4]:
X = data.x
Y = data.y
#X = data[:,:-1]
#Y = data[:, -1].reshape(-1,1)

print(X)
print(Y)

ind = np.arange(X.shape[0])
print(ind)
prop = 0.9
n = int(X.shape[0] * prop)

[[ -2.0757    -0.059627  -0.071558 ...  -0.083049  -0.21885   -0.095476]
 [ 13.924     -0.059627  -0.071558 ...  -0.083049  -0.21885   -0.095476]
 [-11.076     -0.059627  -0.071558 ...  -0.083049   0.031146  -0.095476]
 ...
 [-25.076     -0.059627  -0.071558 ...  -0.083049   0.031146  -0.095476]
 [-23.076     -0.059627  -0.071558 ...  -0.083049  -0.21885   -0.095476]
 [  1.9243    -0.059627  -0.071558 ...  -0.083049   0.031146  -0.095476]]
[[-35.275 ]
 [-26.918 ]
 [  9.9046]
 ...
 [ -5.0114]
 [-38.447 ]
 [  6.4794]]
[    0     1     2 ... 53497 53498 53499]


In [5]:
from gpmaniflow.models.BezierProcess2 import BezierProcess
from gpflow.likelihoods import Gaussian
from gpflow.models import SGPR
from gpflow.inducing_variables import InducingPoints
import gpflow

import tensorflow as tf
import tensorflow_probability as tfp

from gpflow.utilities import print_summary

2022-05-12 08:27:31.744689: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-12 08:27:31.744727: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
BezierRMSE = np.zeros(20)
BezierRMedSE = np.zeros(20)
BezierTestLL = np.zeros(20)


ITERATIONS = 1
ORDER = 5
BATCH_SIZE = 10
INPUT_DIM = X.shape[1]

for i in range(20):
    print('Split', i + 1, 'of', 20)
    np.random.seed(666 + i) # The devil's seed
    np.random.shuffle(ind)
    
    ### GET TRAINING SPLIT AND PREPROCESS ###
    
    Xtrain = X[ind[:n], :]; Ytrain = Y[ind[:n], :]
    Xtest = X[ind[n:], :]; Ytest = Y[ind[n:], :]
    
    mi, ma = Xtrain.min(axis=0), Xtrain.max(axis=0)
    mami = ma - mi
    mami = np.where(mami > 1e-6, mami, 1.) # Don't divide by zero
    Xtrain = (Xtrain - mi) / mami
    Xtest = (Xtest - mi) / mami

    mi = 0.01
    ma = 0.99
    mami2 = np.where(mami > 1e-6, mami, 0.)
    Xtrain = Xtrain * (ma - mi) + mi
    Xtest = Xtest * (ma - mi) + mi

    meany, stdy = np.average(Ytrain), np.std(Ytrain)
    Ytrain = (Ytrain - meany) / stdy
    ############## BEZIER GP ##################
    
    m = BezierProcess(input_dim = INPUT_DIM, orders = ORDER, likelihood = Gaussian(), num_data = Xtrain.shape[0])
    m.likelihood.variance.assign(1.0)
    gpflow.set_trainable(m.likelihood.variance, False)
    
    minibatch_size = BATCH_SIZE

    train_dataset = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain)).repeat().shuffle(Xtrain.shape[0])
    train_iter = iter(train_dataset.batch(minibatch_size))

    training_loss = m.training_loss_closure(train_iter, compile=True)
    optimizer = tf.optimizers.Adam(learning_rate = 0.001)

    @tf.function
    def optimization_step():
        optimizer.minimize(training_loss, m.trainable_variables)

    iterations = ITERATIONS
    print('Training....')
    for step in range(iterations):
        optimization_step()
        if step % 500 == 0:
            elbo = -training_loss().numpy()
            print('Split:', i + 1, 'Step:', step, 'Elbo:', elbo)
    
    gpflow.set_trainable(m.BN, False) # FIX VARIATIONAL PARAMS
    gpflow.set_trainable(m.likelihood.variance, True) # AND TRAIN ONLY LIKELIHOOD VARIANCE
    
    training_loss = m.training_loss_closure(train_iter, compile=True)
    optimizer = tf.optimizers.Adam(learning_rate = 0.01)

    @tf.function
    def optimization_step():
        optimizer.minimize(training_loss, m.trainable_variables)
    for step in range(iterations):
        optimization_step()
        if step % 500 == 0:
            elbo = -training_loss().numpy()
            print('Split:', i + 1, 'Step:', step, 'Elbo:', elbo)
    
    preds, predvar = m.predict_f(Xtest)
    RMSE = np.sqrt( ( (Ytest - (meany + stdy*preds.numpy()))**2).mean())
    print('RMSE:', RMSE)
    RMedSE = np.sqrt( np.median( (Ytest - (meany + stdy*preds.numpy()))**2))
    print('RMedSE:', RMedSE)
    pred_mean, pred_var = m.predict_y(Xtest)
    pred_dist = tfp.distributions.Normal(loc=meany + pred_mean*stdy, scale=stdy*pred_var**0.5)
    loglik = tf.reduce_mean(pred_dist.log_prob(Ytest)).numpy()
    print('Test log-likelihood:', loglik)
    ### Append results
    BezierRMSE[i] = RMSE
    BezierRMedSE[i] = RMedSE
    BezierTestLL[i] = loglik
    
print('Experiment concluded!')

Split 1 of 20


2022-05-12 08:27:34.620271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 08:27:34.620309: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 08:27:34.620342: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martinj): /proc/driver/nvidia/version does not exist
2022-05-12 08:27:34.621253: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-12 08:27:35.220408: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but 

Training....


KeyboardInterrupt: 

In [ ]:
preds, predvar = m.predict_f(Xtest)
RMSE = np.sqrt( ( (Ytest - (meany + stdy*preds.numpy()))**2).mean())
print('RMSE:', RMSE)
RMedSE = np.sqrt( np.median( (Ytest - (meany + stdy*preds.numpy()))**2))
print('RMedSE:', RMedSE)
pred_mean, pred_var = m.predict_y(Xtest)
pred_dist = tfp.distributions.Normal(loc=meany + pred_mean*stdy, scale=stdy*pred_var**0.5)
loglik = tf.reduce_mean(pred_dist.log_prob(Ytest)).numpy()
print('Test log-likelihood:', loglik)

In [ ]:
print('Iterations:', ITERATIONS)
print('Order:', ORDER)
print('Batch size:', BATCH_SIZE)
print('Average RMSE:', np.mean(BezierRMSE), 'Standard deviation:', np.std(BezierRMSE))
print('Average RMedSE:', np.mean(BezierRMedSE), 'Standard deviation:', np.std(BezierRMedSE))
print('Average test-ll:', np.mean(BezierTestLL), 'Standard deviation:', np.std(BezierTestLL))

In [ ]:
BezierRMSE

In [ ]:
Xtest.max()